# Merolagani

> use this for getting floorsheet of a particular day

In [7]:
from selenium import webdriver
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import sys

In [8]:
def search(driver, date):
    """
    Date in mm/dd/yyyy
    """
    driver.get("https://merolagani.com/Floorsheet.aspx")
    date_input = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[1]/div[4]/input')
    search_btn = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[2]/a[1]')
    date_input.send_keys(date)
    search_btn.click()
    if driver.find_elements_by_xpath("//*[contains(text(), 'Could not find floorsheet matching the search criteria')]"):
        print("No data found for the given search.")
        print("Aborting script ......")
        sys.exit()

In [3]:
def get_page_table(driver, table_class):
    soup = BeautifulSoup(driver.page_source,'html')
    table = soup.find("table", {"class":table_class})
    tab_data = [[cell.text.replace('\r', '').replace('\n', '') for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    return df

In [4]:
def scrape_data(driver, date):
    start_time = datetime.now()
    search(driver, date = date)
    df = pd.DataFrame()
    while True:
        page_table_df = get_page_table(driver, table_class="table table-bordered table-striped table-hover sortable")
        df = df.append(page_table_df, ignore_index = True)
        try:
            next_btn = driver.find_element_by_link_text('Next')
            driver.execute_script("arguments[0].click();", next_btn)
        except NoSuchElementException:
            break
    print(f"Time taken to scrape: {datetime.now() - start_time}")    
    return df

In [5]:
def clean_df(df):
    new_df = df.drop_duplicates(keep='first') # Dropping Duplicates
    new_header = new_df.iloc[0] # grabing the first row for the header
    new_df = new_df[1:] # taking the data lower than the header row
    new_df.columns = new_header # setting the header row as the df header
    new_df.drop(["#"], axis=1, inplace=True)
    new_df["Rate"] = new_df["Rate"].apply(lambda x:float(x.replace(",", ""))) # Convert Rate to Float
    new_df["Amount"] = new_df["Amount"].apply(lambda x:float(x.replace(",", ""))) # Convert Amount to Float
    return new_df

In [6]:
options = Options()
options.headless = True
driver = webdriver.Chrome(chrome_options=options) # Start Browser

date = datetime.today().strftime('%m/%d/%Y') # Get today's date
search(driver, date) # Search the webpage
df = scrape_data(driver, date) # Scraping
final_df = clean_df(df) # Cleaning

/home/suyog/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


No data found for the given search.
Aborting script ......


SystemExit: 

/home/suyog/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
final_df.head()

In [ ]:
file_name = date.replace("/", "_")
final_df.to_csv(f"data/{file_name}.csv", index=False) # Save file

# Nepalweb

> use this to get data form nepalweb

In [13]:
import requests
import json
import pandas as pd
from fake_useragent import UserAgent
import time

In [2]:
ua = UserAgent()
headers = {"User-Agent": ua.random}
url = 'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page=0&size=500&sort=contractId,desc'
r = requests.get(url, allow_redirects=True,headers=headers)

In [3]:
data = json.loads(r.content)

In [4]:
floorsheet = data['floorsheets']['content']

In [5]:
df = pd.DataFrame(floorsheet)

In [6]:
start_time = time.time()
i = 0
df = pd.DataFrame()


while True:
    headers = {"User-Agent": ua.random}
    url = f'https://newweb.nepalstock.com.np/api/nots/nepse-data/floorsheet?page={i}&size=500&sort=contractId,desc'
    try:
        r = requests.get(url, allow_redirects=True,headers=headers)
        data = json.loads(r.content)
        floorsheet = data['floorsheets']['content']
        if len(floorsheet)==0:
            break
        df_i = pd.DataFrame(floorsheet)
        df = df.append(df_i)
        i+=1
        time.sleep(0.2)
    except:
        time.sleep(1)
        continue
        
print("--- %s seconds ---" % (time.time() - start_time))

--- 27.05197548866272 seconds ---


In [10]:
df = df.drop(['id', 'contractType'], axis=1)

In [12]:
df.to_csv('data.csv', index=False)

# For saved data

In [89]:
df = pd.read_csv('../django_app/data.csv')

In [90]:
df.head()

,contractId,stockSymbol,buyerMemberId,sellerMemberId,contractQuantity,contractRate,contractAmount,businessDate,tradeBookId,stockId,buyerBrokerName,sellerBrokerName,tradeTime,securityName
0,2021060205000158,SHL,25,58,100,230.0,23000.0,2021-06-02,30661551,147,Sweta Securities Private Limited,Naasa Securities Co. Ltd.,2021-06-02T12:05:38.127605,Soaltee Hotel Limited
1,2021060205000157,SHL,34,58,900,230.0,207000.0,2021-06-02,30661550,147,Vision Securities Pvt. Ltd,Naasa Securities Co. Ltd.,2021-06-02T12:05:38.127178,Soaltee Hotel Limited
2,2021060205000156,SHL,34,58,2300,230.0,529000.0,2021-06-02,30661419,147,Vision Securities Pvt. Ltd,Naasa Securities Co. Ltd.,2021-06-02T12:05:22.05489,Soaltee Hotel Limited
3,2021060205000155,SHL,45,58,700,230.0,161000.0,2021-06-02,30661417,147,Imperial Securities Company Pvt. Ltd.,Naasa Securities Co. Ltd.,2021-06-02T12:05:22.054327,Soaltee Hotel Limited
4,2021060205000154,SHL,42,45,120,232.0,27840.0,2021-06-02,30659778,147,Sani Securities Company Ltd.,Imperial Securities Company Pvt. Ltd.,2021-06-02T12:03:04.744962,Soaltee Hotel Limited


In [91]:
df['tradeTime'] = pd.to_datetime(df['tradeTime']).astype(str)

In [94]:
type(df.loc[0].to_dict()

dict

In [107]:
for i in range(len(df)):
    print(type(df.loc[i].to_dict()))
    break

<class 'dict'>


In [103]:
len(df)

26504